# Análisis y Predicción de Ventas en una Tienda de Retail (Core)

In [ ]:
# Importar pandas
import pandas as pd
# Ruta de archivo CSV
ruta = pd.read_csv('/content/retail_sales_dataset.csv')
# Convertir a DataFrame
df_retail = pd.DataFrame(ruta)
# Visualizar los 5 primeros registros
df_retail.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


**1- Transformación de Datos**
* Crea nuevas columnas: Basándonos en los datos existentes, crea nuevas columnas que sean útiles para el análisis. Por ejemplo, calcula el ingreso total por venta y normaliza las ventas.
* Clasifica los datos: Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’).

In [ ]:
# Normalización de Venta ya que el Dataset ya cuenta con el campo Total Amount

# Calculo del valor máximo 'Importe total'
max_importe = df_retail['Total Amount'].max()
# Calculo del valor mínimo de 'Importe total'
min_importe = df_retail['Total Amount'].min()
# Normalizar la columna 'Importe total'
df_retail['Normalized amount'] = df_retail['Total Amount'].apply(lambda x: (x - min_importe) / (max_importe - min_importe))
# Convertir 'Importe total' a tipo flotante y redondear a 10 decimales
df_retail['Normalized amount'] = df_retail['Normalized amount'].map(lambda x: f"{x:.5f}")
# Mostrar las primeros 10 filas
df_retail.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Normalized amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,0.06329
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,0.49367
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,0.00253
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,0.24051
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,0.03797


In [ ]:
# Clasifica los datos

# Crear un campo Sales Category para clasificar las ventas en categorías según el Total Amount
df_retail['Sales Category'] = df_retail['Total Amount'].apply(lambda x: 'Baja' if x <= 500 else 'Media' if x <= 1000 else 'Alta')
# Contar cuantos registros contiene cada categoría
df_retail['Sales Category'].value_counts()

,count
Sales Category,
Baja,701
Alta,153
Media,146


**2- Agrupación y Agregación**
* Agrupación por múltiples columnas: Realiza agrupaciones por categorías como producto y tienda, producto y mes, etc.
* Aplicar funciones de agregación: Utiliza funciones como sum, mean, count, min, max, std, y var para obtener estadísticas descriptivas de cada grupo.

In [ ]:
# Agrupación por múltiples columnas

# Agrupación por Categoría de Productos y Categoría de Ventas y mostrar la suma del Importe total
df_group = df_retail.groupby(['Product Category', 'Sales Category'])['Total Amount'].sum()
# Mostrar el resultado
df_group

Product Category  Sales Category
Beauty            Alta              74300
                  Baja              26415
                  Media             42800
Clothing          Alta              80500
                  Baja              33380
                  Media             41700
Electronics       Alta              83000
                  Baja              32605
                  Media             41300
Name: Total Amount, dtype: int64

In [ ]:
# Aplicar funciones de agregación

# Aplicar múltiples funciones de agregación a la agrupación por Categoría de Productos y Categoría de Ventas en base a Importe total
df_agg = df_retail.groupby(['Product Category', 'Sales Category'])['Total Amount'].agg(['sum', 'mean', 'count', 'min', 'max', 'std', 'var'])
# Mostrar el resultado
df_agg

sum         mean  count   min   max  \
Product Category Sales Category                                          
Beauty           Alta            74300  1547.916667     48  1200  2000   
                 Baja            26415   125.785714    210    25   500   
                 Media           42800   873.469388     49   600  1000   
Clothing         Alta            80500  1548.076923     52  1200  2000   
                 Baja            33380   134.056225    249    25   500   
                 Media           41700   834.000000     50   600  1000   
Electronics      Alta            83000  1566.037736     53  1200  2000   
                 Baja            32605   134.731405    242    25   500   
                 Media           41300   878.723404     47   600  1000   

                                        std            var  
Product Category Sales Category                             
Beauty           Alta            305.961744   93612.588652  
                 Baja            120.224976   14454.044771  
                 Media           155.155178   24073.129252  
Clothing         Alta            340.984954  116270.739065  
                 Baja            127.772530   16325.819407  
                 Media           159.859632   25555.102041  
Electronics      Alta            348.592442  121516.690856  
                 Baja            126.879358   16098.371541  
                 Media           144.366939   20841.813136

**3- Análisis Personalizado con apply**
* Función personalizada: Aplica funciones personalizadas para realizar análisis específicos que no se pueden lograr con las funciones de agregación estándar.
* Ejemplo de uso avanzado: Calcula la desviación de cada venta respecto a la media de su grupo.

In [ ]:
# Funciones personalizadas para realizar análisis específicos con apply

# Calcular la media por categoría de producto
medias_por_categoria = df_retail.groupby('Product Category')['Total Amount'].mean()
# Crear una nueva columna que calcule la desviación respecto a la media de su categoría
df_retail['Desviación de la Media'] = df_retail.apply(lambda row: row['Total Amount'] - medias_por_categoria[row['Product Category']], axis=1)
# Visualizar las columnas de interés
df_retail[['Product Category', 'Total Amount', 'Desviación de la Media']].head(10)


,Product Category,Total Amount,Desviación de la Media
0,Beauty,150,-317.475570
1,Clothing,1000,556.752137
2,Electronics,30,-428.786550
3,Clothing,500,56.752137
4,Beauty,100,-367.475570
5,Beauty,30,-437.475570
6,Clothing,50,-393.247863
7,Electronics,100,-358.786550
8,Electronics,600,141.213450
9,Clothing,200,-243.247863
